In [1]:
import tkinter as tk
from tkinter import ttk
from project import generate_dict_from_class
from sixgill.definitions import Parameters
from widgets import DualCascadeListBox

In [ ]:
root = tk.Tk()
root.title("Main Application")
root.geometry("300x200")


def open_dual_cascade_listbox():
    DualCascadeListBox(
        root, "Dual Cascade Listbox", generate_dict_from_class(Parameters)
    )


open_button = ttk.Button(
    root, text="Open Dual Cascade ListBox", command=open_dual_cascade_listbox
)
open_button.pack(pady=20)

root.mainloop()